In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

In [2]:
def get_etf():
    etf_csv=pd.read_csv('Developed Asia Pacific ETF List (108).csv') #讀資料夾底下的csv
    symbol=etf_csv['Symbol']#all etf
    inception=etf_csv['Inception'] # all etf inception time
    count=0
    real_symbol=[] #etf
    in_s=[] #time
    for candiate in inception:
        s=datetime.datetime.strptime(candiate, "%d/%m/%Y")
        if s<=datetime.datetime.strptime("31/12/2015", "%d/%m/%Y"):
           # print(s.strftime("%d/%m/%Y"))
            in_s.append(s.strftime("%d/%m/%Y"))
            real_symbol.append(symbol[count])
        count=count+1
    return real_symbol

In [3]:
def crawler(ETF,startDate,endDate,freq,target):
    dict={}
    date_arr=[]
    df = pd.DataFrame()
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/"+ETF+"?symbol="+ETF+"&period1="+str(startDate)+"&period2="+str(endDate)+"&interval="+freq+"&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=HLYrppuGGqQ&corsDomain=finance.yahoo.com&fbclid=IwAR298rNC7s-eA4POkUD5sVlKNEB2YuS4zL9WN23CPyO8CzyUYAF8ezIT9rM"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    if (python_dict['chart']['result'] == None):
        print("已下市")
    else:
        timestamp=python_dict['chart']['result'][0]['timestamp']
        for time in timestamp:
            date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
            dict['Date']=date_arr
        dict[ETF]=python_dict['chart']['result'][0]['indicators'][target][0][target]
        if(freq == '1wk' or freq == '1mo'):
            dict['Date'].pop() #移除最後一個
            dict[ETF].pop() #移除最後一個
        df = pd.DataFrame.from_dict(dict)
        df.set_index("Date" , inplace=True)
    return df

In [4]:
etfs=get_etf()
etfs.remove('ZJPN')
etfs.remove('PAF')# 移除不合規定的etf
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))



def get_Date(freq,target,etfs):
    print('----------'+freq+'----------')
    all_data = pd.DataFrame()
    for Symbol in etfs:
        print('start crawling :', Symbol)
        data = crawler(Symbol,start_time,now,freq,target)
        datalen = data.shape[0]
        if datalen == 0:
            continue
        if datalen > all_data.shape[0]:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[data.index])
        else:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[all_data.index])
    return all_data

In [5]:
all_data_week = get_Date('1wk','adjclose',etfs)
all_data_month = get_Date('1mo','adjclose',etfs)

----------1wk----------
start crawling : EWJ
start crawling : EWY
start crawling : FNDF
start crawling : VPL
start crawling : DXJ
start crawling : EWT
start crawling : EWH
start crawling : EPP
start crawling : FNDC
start crawling : DLS
start crawling : JPIN
start crawling : PXF
start crawling : GSIE
start crawling : EWA
start crawling : FDT
start crawling : IPAC
start crawling : GWX
start crawling : DWM
start crawling : DFJ
start crawling : HEWJ
start crawling : DBJP
start crawling : EWS
start crawling : DOL
start crawling : HFXI
start crawling : PDN
start crawling : IVLU
start crawling : IDOG
start crawling : DTH
start crawling : DIM
start crawling : DOO
start crawling : SCJ
start crawling : ENZL
start crawling : IDLB
start crawling : FJP
start crawling : JPXN
start crawling : IVAL
start crawling : DXJS
start crawling : ISCF
start crawling : DEEF
start crawling : IMOM
start crawling : JPN
start crawling : IDHQ
start crawling : FPA
start crawling : DVYA
start crawling : JPMV
start craw

In [6]:
all_data_week=all_data_week.dropna()
#all_data_week

In [7]:
#all_data_month

In [8]:
all_data_month=all_data_month.dropna()
#all_data_month

In [9]:
def  calculate_ra_1(etf): # calculate one ra
    today_etf=etf[-1]
    return [(today_etf-etf[i])/etf[i] for i in range(0,len(etf)-1)]

In [10]:
def calculate_ra_all(date,etfs,data):
    dict={}
    for etf in etfs:
        dict[etf]=calculate_ra_1(data[etf])
    
    df = pd.DataFrame.from_dict(dict)
    return df

In [13]:
returns=calculate_ra_all(all_data_month.index,etfs,all_data_month)

,EWJ,EWY,FNDF,VPL,DXJ,EWT,EWH,EPP,FNDC,DLS,...,HFXJ,EZJ,ISZE,SCIJ,FHK,HJPX,SCIX,FKO,FAUS,HAUD
0,0.240612,0.348672,0.323711,0.342096,0.144242,0.523363,0.616359,0.475556,0.323501,0.318936,...,0.263717,0.339626,0.293627,0.234725,0.515152,0.295901,0.439127,0.091797,0.321388,0.392645
1,0.310105,0.381962,0.360447,0.390904,0.307549,0.462855,0.605637,0.484789,0.343888,0.333523,...,0.303141,0.460890,0.293627,0.269124,0.487276,0.336656,0.395401,0.070148,0.391344,0.420284
2,0.250398,0.218963,0.263847,0.294016,0.245028,0.355200,0.469194,0.331086,0.237540,0.228245,...,0.250302,0.341204,0.224123,0.214279,0.394172,0.268436,0.284856,-0.026780,0.225113,0.370111
3,0.249303,0.232261,0.220017,0.285226,0.324092,0.438213,0.468451,0.306763,0.212645,0.214684,...,0.208859,0.360614,0.190641,0.204549,0.349955,0.312136,0.264252,-0.010869,0.225409,0.303601
4,0.210097,0.293923,0.238204,0.271714,0.239922,0.408026,0.470682,0.335854,0.196885,0.197177,...,0.278408,0.273852,0.190641,0.172146,0.352975,0.266792,0.291286,0.032422,0.241940,0.275300
5,0.240612,0.236520,0.259588,0.280614,0.403305,0.336873,0.483451,0.331765,0.228370,0.287856,...,0.324767,0.334567,0.190641,0.177610,0.324620,0.386452,0.254196,0.020244,0.186752,0.311849
6,0.180693,0.156766,0.217038,0.200844,0.321491,0.249705,0.362172,0.230051,0.163696,0.189694,...,0.266297,0.211078,0.188187,0.155923,0.238973,0.345240,0.169928,-0.008770,0.095947,0.253641
7,0.159541,0.142602,0.205265,0.198607,0.245933,0.249705,0.353752,0.260251,0.167384,0.187306,...,0.234654,0.173496,0.157034,0.141008,0.214109,0.373801,0.182898,-0.029369,0.105303,0.240864
8,0.135499,0.107614,0.186141,0.168335,0.254354,0.196374,0.293476,0.214050,0.131912,0.156545,...,0.206341,0.137671,0.153396,0.090517,0.175031,0.298945,0.184577,-0.048354,0.127402,0.258123
9,0.127408,0.163245,0.191815,0.175024,0.190736,0.191819,0.319861,0.251036,0.147584,0.173597,...,0.188978,0.135151,0.182714,0.094128,0.215917,0.237591,0.227615,0.023822,0.160767,0.245148


In [14]:
week_returns=calculate_ra_all(all_data_week.index,etfs,all_data_week)
#week_returns

In [15]:
# returns=calculate_ra_all(all_data_month.index,etfs)
# returns

In [16]:
def OmegaSharpeRatio(R, L):
    L=L/100
    n = len(R)
    upside = 0.0
    downside = 0.0
    
    for i in range(n):
        if R[i] < L:
            downside = downside + (L-R[i])
        else:
            upside = upside + (R[i] - L)
            
    if downside != 0:
        OmegaRatio = (upside / downside)-1
    else:
        OmegaRatio = "undefined"
        
    return OmegaRatio

In [17]:
def ranking(etfs,returns,date):
    arr=[]
    symbols=[]
    if date=='week':
        rf=2.24/4
    else:
        rf=2.24
    for etf in etfs:
        if OmegaSharpeRatio(returns[etf],rf)!="undefined":
            arr.append(OmegaSharpeRatio(returns[etf],rf))
            symbols.append(etf)
    dict_omega={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_omega)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df

In [18]:
data=ranking(etfs,week_returns,'week')
data

,ETF_ Symbol,Value
Ranking,,
1,HAUD,2883.096131
2,EWH,817.184182
3,EPP,74.793238
4,SCIX,29.360763
5,EWA,25.142955
6,DOO,21.003610
7,FAUS,16.532639
8,IDHQ,13.957150
9,HFXI,13.154458


In [19]:
data=ranking(etfs,returns,'month')
data

,ETF_ Symbol,Value
Ranking,,
1,EWH,420.559737
2,HAUD,315.560366
3,EPP,44.908598
4,SCIX,26.698052
5,EWA,14.572445
6,DOO,5.288793
7,EWT,4.671522
8,FHK,4.642566
9,AUSE,4.261590


In [ ]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASSR(etf, returns, b3):
    x = np.asarray(returns[etf])
    SR = mean(x)/np.std(x)
    S = skew(x)           #偏度
    ASSR = SR * cmath.sqrt(1+b3*((S/3)*SR))
    
    return ASSR

In [20]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASKSR(etf, returns, rf):
    x = np.asarray(returns[etf])
    u = mean(x)/x.size    #期望值
    v = var(x, ddof=1)    #變異數
    SD = math.sqrt(v)     #標準差
    S = skew(x)           #偏度
    K = kurtosis(x)       #峰度

    c1 = 3*K-4*pow(S,2)-9
    c2 = 3*K-5*pow(S,2)-9
    a = ( 3*(cmath.sqrt(c1) ) / ( v*c2 ) )
    b = ( 3*S ) / ( SD*c2 )
    n = u - ( (3*S*SD) / c1 )
    d = 3*SD * ( (cmath.sqrt(c2)) / (c1) ) 
    phi = cmath.sqrt(pow(a,2)*pow(b,2))
    a_ = b + ( a*(n-rf) / cmath.sqrt(pow(d,2)+pow(n-rf,2))) 
    ASKSR = cmath.sqrt(2*(a_*(n-rf)-d*(phi-cmath.sqrt(pow(a,2)-pow(b-a_,2)))))
    
    return ASKSR

In [21]:
def rankingByASKSR(etfs, returns):
    arr=[]
    symbols=[]
    for etf in etfs:
        if sharpeASKSR(etf, returns ,0)!="undefined":
            if sharpeASKSR(etf, returns ,0).imag == 0:
                arr.append(sharpeASKSR(etf, returns ,0))
                symbols.append(etf)
            else:
                arr.append(sharpeASSR(etf, returns ,0))
                symbols.append(etf)
    dict_ASKSR={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_ASKSR)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df

In [22]:
#ASKSR -- Monthly Data
rankingByASKSR(etfs, returns)

,ETF_ Symbol,Value
Ranking,,
1,FKO,(2.94988907403+0j)
2,FPA,(2.68990857978+0j)
3,FDTS,(2.65310899501+0j)
4,IMOM,(2.65191121661+0j)
5,FJP,(2.5174669761+0j)
6,EWV,(2.42415919056+0j)
7,ISCF,(2.25894053451+0j)
8,DEEF,(2.21051804121+0j)
9,FDT,(2.19889671738+0j)


In [23]:
#ASKSR -- Weekly Data
rankingByASKSR(etfs, week_returns)

,ETF_ Symbol,Value
Ranking,,
1,IMOM,(2.94278303481+0j)
2,FPA,(2.56903164014+0j)
3,DEEF,(2.52654014917+0j)
4,FDTS,(2.51662274356+0j)
5,EWV,(2.47864868493+0j)
6,FKO,(2.4341357648+0j)
7,ISCF,(2.30513288786+0j)
8,FJP,(2.16970489602+0j)
9,FDT,(2.16542224055+0j)


In [24]:
import numpy as np
import math
from scipy.optimize import fsolve,leastsq,root

def global_index_riskness(returns):
    x = np.asarray(returns)
    g = mean(x)/x.size
    n = len(returns)
    def f(alpha):
        sum=0
        for i in range(len(returns)):
            sum=np.exp(-returns[i]*alpha)+sum
        #print("sum",sum,len(returns))
        return sum-len(returns)
    if g == 0:
        minus_alphaG = 0
    elif g < 0:
        minus_alphaG = -1*fsolve(f,-10)
    else:
        minus_alphaG = -1*fsolve(f,10)
        
    riskiness = math.exp(minus_alphaG)
    return riskiness

In [25]:
def ranking_riskiness(etfs, returns):
    arr=[]
    symbols=[]
    
    for etf in etfs:
        if global_index_riskness(returns[etf]):
            arr.append(global_index_riskness(returns[etf]))
            #print('-----' + str(etf) + '-----')
            symbols.append(etf)
    dict_riskness={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_riskness)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df

In [26]:
#Global Index riskiness -- Week Data
data=ranking_riskiness(etfs,week_returns)
data

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


,ETF_ Symbol,Value
Ranking,,
1,FKO,1.305059e+05
2,EWV,2.425779e+02
3,DXJF,6.011521e+01
4,KORU,8.530735e+00
5,FJP,3.339288e+00
6,JPNL,1.314906e+00
7,IMOM,1.016616e+00
8,ENZL,1.000000e+00
9,EWA,1.000000e+00


In [27]:
#Global Index riskiness -- Month Data
data=ranking_riskiness(etfs,returns)
data

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


,ETF_ Symbol,Value
Ranking,,
1,FKO,1.547051e+09
2,EWV,6.361299e+02
3,DXJF,6.554446e+01
4,KORU,4.599388e+01
5,FJP,1.525683e+01
6,IMOM,4.185146e+00
7,JPNL,1.144999e+00
8,DOO,1.000000e+00
9,EWA,1.000000e+00
